# Estudo de Caso: OpenStreetMap

**Área: Brasília, DF, Brasíl**

* https://www.openstreetmap.org/relation/2758138

* http://overpass-api.de/api/map?bbox=-48.2815,-15.9514,-47.5914,-15.4854


**Motivação**: Este Mapa representa a cidade onde eu moro, gostaria de estudar sobre os dados da cidade e por morar aqui é mais fácil detectar erros no conjunto de dados.

# Problemas Encontrados

Após Gerar um Arquivo de Treino utilizando o código abaixo e realizando uma auditoria poucos problemas foram encontrados que serão discutidos em tópicos separados:

* Enderecos muito abreviados()

* CEP(Código postal) em diferentes formatos

# * Endereços muito abreviados

Fazendo uma auditoria direto no arquivo OSM antes da importação para o banco de dados SQL foram encontrados diversos endereços com várias abreviações como QD, ED, Lt etc... Apesar de brasília ser conhecido por endereços abreviados como SHIN QL 9 11, esticar todo o endereço seria estranho para os próprios habitantes e ruim para o banco de dados, então foi decidido esticar apenas as abreviações não referentes ao inicio do endereço, traduzindo apenas QD para Quadra, Av para Avenida etc...

Exemplo: **SHIN QL 9 11 => SHIN Quadra do Lago 9 11**

# * Códigos postais em diferentes formatos

**Auditando Código-postal(CEP)**

   O Padrão estabelecido para um formato de CEP para o banco de dados é somente texto com 8 digitos numéricos, Foram identificados alguns registros fora do padrão de 8 dígitos, foi auditado a validade dos CEPs para a região e se existiam coordenadas(latitude,longitude) com CEPs diferentes, nenhum erro de validação e consistência foi encontrado. O Código em questão utilizado para a auditória foi o **audit_cep.py**

In [ ]:
Encontrados 15 CEPs Com Problemas
set(['70.687-305',
     '70070-600',
     '70070-940',
     '70238-110',
     '70419-900',
     '70716-900',
     '70867-510',
     '70867-540',
     '71.961-540',
     '71503-505',
     '71570-613',
     '71572-304',
     '71680-357',
     '71925-720',
     '72220-061'])

# Gerando XML corrigido

Apos tomar ciência dos problemas acima foi escrito código para gerar um XML com a correção proposta, tanto para as abreviações dos endereços quanto para os códigos postais, o código está no arquivo **osm_cleaned_generator.py**.

Feito Isto, foi executado o código contigo no arquivo data.py para criação dos arquivos CSV e assim importar os dados para um banco de dados SQL.

# Análise de dados

Esta seção contém números e estatísticas adquiridos por consultas no banco de dados além de ideias adicionais. Todas as Queries aqui listadas foram consultadas no Banco de Dados gerado a partir da importação dos CSVs que foram gerados do arquivo bsb.cleaned.osm

**Tamanho dos Arquivos**



In [ ]:
bsb.osm ................. 79.2 MB
bsb_cleaned.osm.......... 79,4 MB
brasilia_osm.db.......... 42 MB
nodes.csv ............... 28 MB
nodes_tags.csv .......... 1 MB
ways.csv ................ 4.5 MB
ways_tags.csv ........... 5.1 MB
ways_nodes.cv ........... 10.2 MB 

** Número de Nodos**

In [ ]:
sqlite> SELECT COUNT(*) FROM nodes;

320216

** Número de 'Ways'**

In [ ]:
sqlite> SELECT COUNT(*) FROM ways;

71143

** Número de usuários únicos**

In [ ]:
#Retirado do Projeto de exemplo: https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md , 
#A Realização desta query por JOIN estava muito lenta, e a partir desta fonte pude verificar que 
#um subSelect melhoraria a performance.
sqlite> SELECT COUNT(DISTINCT(e.uid))          
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;

545

** Quantidade de locais para bicicletas**

In [ ]:
sqlite> SELECT COUNT(*) as num FROM nodes_tags WHERE key='bicycle_parking' ORDER BY num DESC;

80

# Idéias Adicionais



# * Crowdsourcing estimulado

A fim de estimular a contribuição é sugerido um esquema que estimulasse o crowdsourcing e ao mesmo tempo a confiabilidade das informações registradas. Um sistema de curtidas como em diversas redes sociais seria implementado caso usuários consultassem a informação e confirmassem como Verídico e condizente com a realidade, este índice já teria um bonus de curtidas iniciais caso a informação registrada fosse de um morador da região local, quanto mais curtidas mais preciso a informação. Para implementar esta solução seria necessário pensar e contornar as seguinte situações:

* Cada "Curtida" deverá ser ser vinculada a um usuário do OSM para evitar mais de uma curtida da mesma pessoa em um mesmo Node ou Way, Sendo assim, só poderá participar usuários com registro.

* Usuários deverão ter em seu registro um código postal(CEP) para que ganhe bônus de curtidas no Node ou Way caso registrasse um de sua região.

* Seria realizado um levantamento de usuários cadastrados que moram na região, caso o número de curtidas superassem 70% do tamanho da população, uma insígnia de "Endereço Confiável" seria atribuido a aquele Registro(podendo ser inclusive uma tag extra) e o usuário poderia ser recompensado de alguma forma.


# Exploração de Dados Adicionais



**Quais os CEPs com maior número de registros registros?**

In [ ]:


sqlite3> SELECT e.value, COUNT(*) as num 
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) as e 
WHERE e.key = 'postcode' 
GROUP BY e.value 
ORDER BY num DESC 
LIMIT 10;

73045151|68
71680357|54
71570613|44
71503505|23
71060230|15
70910900|13
71503502|11
71681740|8
71680379|7
71503501|6

Segundo a API dos correios o CEP com maior número de registros(73045151) é de Sobradinho, uma região administrativa do Distrito Federal, é uma informação de certa forma surpreendente.

**Quais as Maiores Opções de Turismo?**

In [ ]:
SELECT value, COUNT(*) as num FROM nodes_tags WHERE key='tourism' GROUP BY value ORDER BY num DESC LIMIT 10;
hotel|60
viewpoint|18
attraction|9
museum|7
picnic_site|7
information|4
caravan_site|3
artwork|2
hostel|2
gallery|1

O Número de Hotéis é maior do que do que todos os restantes dos registros, acredito que se isso for investigado mais a fundo poderia dar uma idéia ou da falta de registros de atrativos no OpenStreetMap ou que realmente faltas mais opções de turismo.

**Horário de Funcionamento mais comum para restaurantes?**

In [ ]:
sqlite3> SELECT nodes_tags.value, COUNT(*) as num
   FROM nodes_tags 
   JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
   ON nodes_tags.id=i.id
   WHERE nodes_tags.key='opening_hours'
   GROUP BY nodes_tags.value
   ORDER BY num DESC
   LIMIT 1;
    
24/7 | 5


O Horário de funcionamento mais comum é o famoso 24/7 ou seja 5 restaurantes aqui registrados estão sempre abertos, boa notícia para quem gosta de comer, mas não se engane se estiver esperando variedade, pois de acordo com a próxima consulta: 

In [ ]:
SELECT nodes_tags.value, COUNT(*) as num
   ...>    FROM nodes_tags 
   ...>    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='24/7') i
   ...>    ON nodes_tags.id=i.id
   ...>    WHERE nodes_tags.key='name'
   ...>    GROUP BY nodes_tags.value
   ...>    ORDER BY num DESC
   ...>    LIMIT 1;
    
Subway|5

As 5 unidades 24/7 encontradas são todos Subway.

# Conclusão

Após esta revisão sobre este conjunto de dados foi identificado pouquissimos problemas em relação as informações registradas, sendo os problemas apenas em relação ao padrão do tipo de dados estabelecido, as informações como CEP e endereços pelo que se pode auditar foram fiéis a realidade e o restante dos endereços e CEPs acredito que foram bem limpos após esta análise, como a maioria do problema foi em relação a uma forma padrão de dados, é sugerido apenas que proponham que certos tipos de dados sigam um padrão estabelecido(Códigos postais sem caracteres especiais por exemplo). 